In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_paths = tf.io.matching_files('d:\\IT\\study\\study\\deeplearning\\final\\CIFAR100\\train\\*.png')
train_paths = train_paths.numpy()

test_paths = tf.io.matching_files('d:\\IT\\study\\study\\deeplearning\\final\\CIFAR100\\test\\*.png')
test_paths = test_paths.numpy()

In [ ]:
classes = []
label_dict = dict()
label_dict_inv = dict()
for path in train_paths:
  path = path.decode()
  label = path[path.find('_')+1:path.find('.png')]
  
  if label not in label_dict:
    label_dict[label] = len(label_dict)

for key in label_dict:
  label_dict_inv[label_dict[key]] = key

print(label_dict)
print(label_dict_inv)

In [ ]:
# train_x_path = []
# train_y = []
# for path in paths:
#   path = path.decode()
#   label = path[path.find('_')+1:path.find('.png')]
#   train_x_path.append(path)
#   train_y.append(label_dict[label])

In [ ]:
def tf_read_image(path):
  gfile = tf.io.read_file(path)
  image = tf.io.decode_image(gfile, dtype=tf.float32)
  return image


def make_batch_x_y(batch_paths):
  batch_images = []
  batch_labels = []
  
  for path in batch_paths:
    path = path.decode()
    image = tf_read_image(path)
    label = label_dict[path[path.find('_')+1:path.find('.png')]]
    batch_images.append(image)
    batch_labels.append(label)
  return tf.convert_to_tensor(batch_images), tf.convert_to_tensor(batch_labels)

In [ ]:
train_x_1d, train_y_1d = make_batch_x_y(train_paths)

In [ ]:
def Mymodel(input_shape, num_classes):
  my_inputs = tf.keras.layers.Input(shape=input_shape)   
  net = tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation='relu')(my_inputs)           
  net = tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation='relu')(net)             
  net = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(net)                                      
  net = tf.keras.layers.Dropout(0.2)(net)                                                    

  net = tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation='relu')(net)            
  net = tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation='relu')(net)            
  net = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(net)                                    
  net = tf.keras.layers.Dropout(0.2)(net)  

  net = tf.keras.layers.Flatten()(net)                                                      
  net = tf.keras.layers.Dense(512, activation='relu')(net)
  # net = tf.keras.layers.Dense(512)(net,  activation='relu')
  net = tf.keras.layers.Dropout(0.3)(net)
  net = tf.keras.layers.Dense(num_classes, activation="softmax")(net)                       
  model = tf.keras.Model(inputs=my_inputs, outputs=net, name="Final_Project")
  return model

In [ ]:
input_shape = (32, 32, 3)       # 높이, 폭, 채널수
num_classes = 100                # 100개로 설정
mymodel = Mymodel(input_shape, num_classes)
mymodel.compile(optimizer=tf.keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mymodel.summary()


In [ ]:
num_epochs = 5          
batch_size = 32
hist = mymodel.fit(train_x_1d, train_y_1d, batch_size = batch_size, shuffle = True, epochs = num_epochs)

In [ ]:
test_x_1d, test_y_1d = make_batch_x_y(test_paths)

results = mymodel.evaluate(test_x_1d, test_y_1d, batch_size=32)
print("test loss, test acc:", results)

In [ ]:
test_img = test_x_1d[6510]
output = mymodel.predict(test_img[tf.newaxis,...])
output = np.argmax(output)

print(label_dict_inv[output])
plt.imshow(test_img)